In [84]:
import numpy as np
import random
import pandas as pd
from random import randint
import math

In [85]:
class Particle_of_swarm(object):
    def __init__(self, hiddenSize, inputSize, outputSize):
        # initiate layers
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize

        layers = [self.inputSize] + self.hiddenSize + [self.outputSize]

        # initiate positions
        positions = []
        for i in range(len(layers)-1):
            p = np.random.uniform(-10,10,(layers[i], layers[i+1]))
            #p = np.random.rand(layers[i], layers[i+1])
            positions.append(p)
        self.positions = positions
        self.positions_best = positions

        velocitys = []
        for i in range(len(layers) - 1):
            v = np.random.uniform(-0.5,0.5,(layers[i], layers[i+1]))
            #v = np.random.rand(layers[i], layers[i+1])
            velocitys.append(v)
        self.velocitys = velocitys

        self.pbest = float('inf')

    def feedForward(self, X):
        Output_node = X
        for i, p in enumerate(self.positions):
            
            # print("i = ",i,len(self.positions))
            # print(Output_node.shape)
            # print(p.shape)
            # print("========================================")
            v = np.dot(Output_node, p)
            Output_node = self.sigmoid(v)

        return Output_node

    def sigmoid(self, s, deriv=False):
        if (deriv == True):
            return s * (1-s)
        return 1/(1 + np.exp(-s))

    def object_funct(self, X, Y):
                    # Random data
        seed = randint(1, 25*100)

        np.random.seed(seed)
        np.random.shuffle(X)

        np.random.seed(seed)
        np.random.shuffle(Y)

        sum_err = 0
        for j, input in enumerate(X):
            target = Y[j]
            output = self.feedForward(input)

            sum_err += self._mae(target, output)

        self.fx = sum_err/len(X) #(1/(sum_err+1))
        return self.fx

    def _mae(self, target, output):
        #print(target - output)
        return np.average(abs(target - output))



In [86]:
def _readfile(file):

    # ----- Clean NaN Values -----
    df = pd.read_csv(file)
    df = df.fillna(method = 'ffill')

    # ----- Create Features -----
    X = df[['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)','T','RH','AH']].copy(deep=False)
    X.drop(X.tail(240).index,inplace=True)
    X['PT08.S1(CO)'] = X['PT08.S1(CO)']/X['PT08.S1(CO)'].max()
    X['PT08.S2(NMHC)'] = X['PT08.S2(NMHC)']/X['PT08.S2(NMHC)'].max()
    X['PT08.S3(NOx)'] = X['PT08.S3(NOx)']/X['PT08.S3(NOx)'].max()
    X['PT08.S4(NO2)'] = X['PT08.S4(NO2)']/X['PT08.S4(NO2)'].max()
    X['PT08.S5(O3)'] = X['PT08.S5(O3)']/X['PT08.S5(O3)'].max()
    X['T'] = X['T']/X['T'].max()
    X['RH'] = X['RH']/X['RH'].max()
    X['AH'] = X['AH']/X['AH'].max()
    
    # ----- Create Desired outputs -----
    label = df[['C6H6(GT)']].copy(deep=False)

    Y_10Day = label.iloc[240:,:].reset_index(drop=True)
    Y_10Day.rename(columns={"C6H6(GT)":"C6H6(GT)_10"}, inplace = True)

    Y_5Day = label.iloc[120:,:].reset_index(drop=True)
    Y_5Day.drop(Y_5Day.tail(120).index,inplace=True)
    Y_5Day.rename(columns={"C6H6(GT)":"C6H6(GT)_5"}, inplace = True)

    Y = pd.concat([Y_5Day,Y_10Day], axis=1)
  
    Y['C6H6(GT)_5'] = Y['C6H6(GT)_5']/Y['C6H6(GT)_5'].max()
    
    Y["C6H6(GT)_10"] = Y["C6H6(GT)_10"]/Y["C6H6(GT)_10"].max()
    Input = X.to_numpy()
    #Output = Y.to_numpy()
    Output = Y['C6H6(GT)_5'].to_numpy()
    return Input,Output

Input,Output = _readfile('data/AirQualityUCI.csv')
#print(Input)


In [87]:
def cross_validations_split(shape,folds):
    fold_size = int(shape * folds/100)
    k = 0
    index = []
    for i in range(1,folds+1):
        if i < folds:
            index.append([k,i*fold_size])
        else:
            index.append([k,shape])
        k = i*fold_size
    return index

In [88]:

particles = []
num_of_particle = 12


for i in range(0, num_of_particle):
    par = Particle_of_swarm([6,4], 8, 2)
    particles.append(par)


gbest = [0,float('inf')]
gbest_position = 0
#-------------------------------------------------------

train_mean_pbest = []
train_gbest = []

test_mean_pbest = []

#-------------------------------------------------------
k=1
for a,b in cross_validations_split(Input.shape[0],10):
    x_train = np.concatenate((Input[:a],Input[b+1:]))
    y_train = np.concatenate((Output[:a],Output[b+1:]))
    x_test = Input[a:b,:]
    y_test = Output[a:b]
    list_fx = []
    print("-------------------- fold : ",k," --------------------")
    k+=1
    for j in range(25):
        for i,p in enumerate(particles):
            #print("----------------------------------")
            #print("--- gbest",gbest[0]," : ",gbest[1])
            fx = p.object_funct(x_train, y_train)
            list_fx.append(fx)
            print("--- fx", i ," : ", fx)

        # check pbest
            if fx < p.pbest:
                print("Update pbest ",i," : ",round(fx,3), " / ",round(p.pbest,3))
                p.pbest = fx
                p.positions_best = p.positions.copy()
                

        #check gbest
            if fx < gbest[1]:
                gbest[1] = fx
                gbest[0] = i
                #p.positions_best = p.positions.copy()
                gbest_position = p.positions.copy()
                print("Update gbest",i," : ",fx)

        # update velocity & position
        for i,p in enumerate(particles):
            for c in range(0, len(p.velocitys)):
                r1=random.random()
                r2=random.random()
                d1 = 1*r1
                d2 = 2*r2
                d = d1+d2
                g = 1 - (1/(d)) + math.sqrt(abs(d**2 - 4*d))/2
                #print("velocity [",i,"] : ",p.velocitys[c])
                p.velocitys[c] = g*(0.2*p.velocitys[c] + (d1 * (p.positions_best[c]-p.positions[c])) + (d2 * (gbest_position[c] - p.positions[c])))
                #print("velocity update [",i,"] : ",p.velocitys[c]) 
                #print("position [",i,"] : ",p.positions[c][0][0])
                p.positions[c] += p.velocitys[c]
                #print("position update [",i,"] : ",p.positions[c][0][0]) 


        print(" Epoch : " , j+1 ," | err : ", sum(list_fx)/len(list_fx) )

    train_mean_pbest.append(sum(list_fx)/len(list_fx))
    train_gbest.append(gbest[1])
    
    list_fx_test = []
    for i,p in enumerate(particles):
        fx = p.object_funct(x_test, y_test)
        list_fx_test.append(fx)
    test_mean_pbest.append(sum(list_fx_test)/len(list_fx_test))
    print("=========== Test ===========")
    print("Error of test  :  ",round(sum(list_fx_test)/len(list_fx_test),6))
    print("Error of train :  ",round(sum(list_fx)/len(list_fx),6))
    print("gbest : ",round(gbest[1],6))





090550931270841
--- fx 0  :  0.09055095103904585
--- fx 1  :  0.09055095103904374
--- fx 2  :  0.09055113063657519
--- fx 3  :  0.09055095103904003
--- fx 4  :  0.09055095103904817
--- fx 5  :  0.09055095103904533
--- fx 6  :  0.09055095103904624
--- fx 7  :  0.09055095103820715
--- fx 8  :  0.09055095103901824
--- fx 9  :  0.09055095103904587
--- fx 10  :  0.09055095104032801
--- fx 11  :  0.0905509510390443
 Epoch :  6  | err :  0.09055093705995759
--- fx 0  :  0.09055095103904652
--- fx 1  :  0.09055095103904635
--- fx 2  :  0.09055112760273672
--- fx 3  :  0.09055095103904727
--- fx 4  :  0.09055095103904368
--- fx 5  :  0.09055095103904662
--- fx 6  :  0.09055095103904585
--- fx 7  :  0.0905509510403876
--- fx 8  :  0.09055095103908234
--- fx 9  :  0.09055095103904649
--- fx 10  :  0.0905509510376782
--- fx 11  :  0.09055095103904427
 Epoch :  7  | err :  0.09055094115891903
--- fx 0  :  0.09055095103904623
--- fx 1  :  0.0905509510390463
--- fx 2  :  0.09055065742509738
--- fx 3 

In [89]:
print(train_mean_pbest)
print(train_gbest)
print(test_mean_pbest)
result = pd.DataFrame({'train_pbest': train_mean_pbest, 'train_gbest': train_gbest,'test_pbest': test_mean_pbest})

[0.22378085523405516, 0.11151970573102372, 0.09395063322951129, 0.09212169800801506, 0.09115113918227552, 0.08624795018549711, 0.08659457198622095, 0.0886162844939438, 0.09055094284839989, 0.09086444231045779]
[0.10822340301509213, 0.08945008364902579, 0.08945008364902579, 0.08945008364902579, 0.08945008364902579, 0.08621076706585991, 0.08621076706585991, 0.08621076706585991, 0.08621076706585991, 0.08621076706585991]
[0.1344349300493622, 0.09864104094041537, 0.08177981566031661, 0.06783804736126947, 0.0764670910801074, 0.12034141560015903, 0.1172300692647556, 0.09901486429535074, 0.08141856684697553, 0.07870153855015406]


In [90]:
result.to_csv('swarm2.csv', encoding='utf-8')

In [91]:
for i,p in enumerate(particles):
    print(i,p.positions[0][0])

0 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
1 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
2 [ 2.58496897 -0.45155606  0.94323455 -1.71363307  9.20924325 -0.14545802]
3 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
4 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
5 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
6 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
7 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
8 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
9 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
10 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
11 [ 2.58496897 -0.45155606  0.94323455 -1.71363308  9.20924325 -0.14545801]
